In [2]:
import numpy as py
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)


In [5]:
image, label = train_data[0]

In [6]:
image.size()

torch.Size([3, 32, 32])

In [7]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [8]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        # First block
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)   #add batch normalization after convulutions to stabilize learning
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.25) #add dropout before fc to reduce overfittingg
        
        # Second block
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout2 = nn.Dropout(0.25)
        
        # Third block
        self.conv5 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(128, 128, 3, padding=1)
        self.bn6 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout3 = nn.Dropout(0.25)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.bn7 = nn.BatchNorm1d(512)
        self.dropout4 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)
        
    def forward(self, x):
        # First block
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        x = self.dropout1(x)
        
        # Second block
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        x = self.dropout2(x)
        
        # Third block
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.pool3(x)
        x = self.dropout3(x)
        
        # Fully connected
        x = torch.flatten(x, 1)
        x = F.relu(self.bn7(self.fc1(x)))
        x = self.dropout4(x)
        x = self.fc2(x)
        
        return x

In [9]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [10]:
for epoch in range(30):
    print(f'training epoch {epoch}...')

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'loss: {running_loss / len(train_loader):.4f}')

training epoch 0...
loss: 1.5178
training epoch 1...
loss: 1.1451
training epoch 2...
loss: 0.9895
training epoch 3...
loss: 0.8900
training epoch 4...
loss: 0.8198
training epoch 5...
loss: 0.7713
training epoch 6...
loss: 0.7223
training epoch 7...
loss: 0.6884
training epoch 8...
loss: 0.6561
training epoch 9...
loss: 0.6258
training epoch 10...
loss: 0.5986
training epoch 11...
loss: 0.5814
training epoch 12...
loss: 0.5555
training epoch 13...
loss: 0.5424
training epoch 14...
loss: 0.5277
training epoch 15...
loss: 0.5087
training epoch 16...
loss: 0.4921
training epoch 17...
loss: 0.4823
training epoch 18...
loss: 0.4652
training epoch 19...
loss: 0.4534
training epoch 20...
loss: 0.4399
training epoch 21...
loss: 0.4313
training epoch 22...
loss: 0.4149
training epoch 23...
loss: 0.4110
training epoch 24...
loss: 0.4000
training epoch 25...
loss: 0.3961
training epoch 26...
loss: 0.3850
training epoch 27...
loss: 0.3713
training epoch 28...
loss: 0.3647
training epoch 29...
los

In [11]:
torch.save(net.state_dict(), 'trained_net.pth')

In [12]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [13]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, pred = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()

accuracy = 100 * correct / total

print(f'Accuracy: {accuracy}%')

Accuracy: 86.4%


In [14]:
base_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(img_path):
    img = Image.open(img_path)
    img = base_transform(img)
    img = img.unsqueeze(0)
    return img

img_path = ['Dog_Breeds.jpg', 'images.jpg', 'Cat03.jpg']
images = [load_image(image) for image in img_path]

net.eval()
with torch.no_grad():
    for img in images:
        output = net(img)
        _, pred = torch.max(output, 1)
        print(f'Prediction: {class_names[pred.item()]}')

Prediction: dog
Prediction: plane
Prediction: cat
